# Tác vụ 2a: Tóm tắt văn bản với các tệp nhỏ bằng Titan Text Premier


Trong sổ tay này, bạn sẽ nhập một chuỗi văn bản nhỏ trực tiếp vào API Amazon Bedrock (sử dụng mô hình Titan Text) và hướng dẫn API này tóm tắt văn bản đầu vào. Bạn có thể áp dụng cách tiếp cận này để tóm tắt bản chép lời cuộc gọi, bản chép lời cuộc họp, sách, bài viết, bài đăng blog và các nội dung liên quan khác khi độ dài văn bản đầu vào nằm trong giới hạn kích thước ngữ cảnh của mô hình.

## Tác vụ 2a.1: Thiết lập môi trường

Trong tác vụ này, bạn sẽ thiết lập môi trường của mình.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Tác vụ 2a.2: Viết câu lệnh với văn bản cần tóm tắt

Trong tác vụ này, bạn sẽ dùng một đoạn văn bản ngắn có ít mã token hơn độ dài tối đa mà mô hình nền tảng hỗ trợ. Bạn sẽ lấy một đoạn trong [bài đăng trên blog AWS] (https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) công bố về Amazon Bedrock để làm văn bản đầu vào mẫu cho bài thực hành này.

Câu lệnh bắt đầu bằng hướng dẫn `Please provide a summary of the following text.` (Vui lòng tóm tắt văn bản sau). 

In [ ]:
prompt_data = """

Please provide a summary of the following text:

AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

## Tác vụ 2a.3: Tạo nội dung yêu cầu với các tham số suy luận và câu lệnh 

Trong tác vụ này, bạn sẽ tạo nội dung yêu cầu với các tham số suy luận và câu lệnh ở trên.

In [ ]:
# request body
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":2048,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

## Tác vụ 2a.4: Gọi mô hình nền tảng qua Boto3

Trong tác vụ này, bạn sẽ gửi một yêu cầu API tới Amazon Bedrock để chỉ rõ các tham số yêu cầu: `modelId`, `accept` và `contentType`. Theo câu lệnh được cung cấp, mô hình nền tảng trong Amazon Bedrock sẽ tóm tắt văn bản đầu vào.

### Tạo đầu ra hoàn chỉnh

Theo mặc định, dịch vụ Amazon Bedrock sẽ tạo toàn bộ bản tóm tắt cho một câu lệnh nhất định trong một lần xuất đầu ra. Quá trình này có thể diễn ra chậm nếu đầu ra của mô hình chứa nhiều mã token. 

In [ ]:
#model configuration and invoke the model
modelId = 'amazon.titan-text-premier-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### Tạo đầu ra truyền trực tuyến

Tiếp theo, bạn sẽ khám phá cách sử dụng API invoke_model_with_response_stream của Amazon Bedrock để truyền trực tuyến các đầu ra của mô hình. Nhờ vậy, người dùng có thể sử dụng đầu ra khi chúng được tạo. Thay vì tạo toàn bộ đầu ra cùng một lúc, API này sẽ trả về một ResponseStream để gửi các đoạn đầu ra nhỏ hơn từ mô hình khi chúng được tạo. Bạn có thể hiển thị các đầu ra truyền trực tuyến này trong chế độ xem liên tục, có thể sử dụng.

In [ ]:
#invoke model with response stream
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

Vậy là bạn đã thử nghiệm việc sử dụng SDK boto3 để truy cập API Amazon Bedrock. SDK này cung cấp quyền truy cập cơ bản vào các khả năng của Bedrock theo lập trình. Bằng việc tận dụng API này, bạn có thể triển khai hai trường hợp sử dụng: 1) Tạo toàn bộ văn bản tóm tắt nội dung tin tức AWS cùng một lúc và 2) Truyền trực tuyến bản tóm tắt theo từng đoạn để tăng khả năng xử lý.

### Tự thực hiện
- Thay đổi câu lệnh theo trường hợp sử dụng cụ thể của bạn và đánh giá đầu ra của các mô hình khác nhau.
- Thử nghiệm với độ dài mã token để nắm được độ trễ và khả năng phản hồi của dịch vụ.
- Áp dụng các nguyên tắc tạo câu lệnh khác nhau để có được đầu ra tốt hơn.

### Dọn dẹp

Bạn đã hoàn thành sổ tay này. Để chuyển sang phần tiếp theo của phòng thực hành, hãy làm như sau:

- Đóng tệp sổ tay này và tiếp tục với **Task2b.ipynb**.